In [55]:
# import libraries (you may add additional imports but you may not have to)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-07-21 11:49:22--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   141MB/s    in 0.2s    

2025-07-21 11:49:22 (141 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [56]:
# consider using other joins, like using mean rating etc. ...
avg_ratings = df_ratings.groupby('isbn')['rating'].mean().reset_index()
df_joined = df_books.merge(avg_ratings, on='isbn', how='left')

print(len(df_books), len(df_ratings), len(df_joined))
df_joined.drop('isbn', axis=1, inplace=True)  # unlikely to hold actual information (?)
df_joined = df_joined.drop_duplicates(subset=['title', 'author'])
df_joined.head()

271379 1149780 271379


,title,author,rating
0,Classical Mythology,Mark P. O. Morford,0.000000
1,Clara Callan,Richard Bruce Wright,4.928571
2,Decision in Normandy,Carlo D'Este,5.000000
3,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,4.272727
4,The Mummies of Urumchi,E. J. W. Barber,0.000000


In [58]:
# transform text columns
vectorizer_title = TfidfVectorizer()
X_title = vectorizer_title.fit_transform(df_joined['title'].fillna('unknown'))

vectorizer_author = TfidfVectorizer()
X_author = vectorizer_author.fit_transform(df_joined['author'].fillna('unknown'))

X_combined = hstack([X_title, X_author])

In [62]:
# add your code here - consider creating a new cell for each section of code
skl_nn = NearestNeighbors(n_neighbors=6)
skl_nn.fit(X_combined)

NearestNeighbors(n_neighbors=6)

In [63]:
query_row = df_joined[df_joined['title'] == 'The Queen of the Damned (Vampire Chronicles (Paperback))'].iloc[0]
X_query_title = vectorizer_title.transform([query_row['title']])
X_query_author = vectorizer_author.transform([query_row['author']])
X_query = hstack([X_query_title, X_query_author])

distances, indices = skl_nn.kneighbors(X_query, return_distance=True)
print(distances, indices)
df_joined['title'].iloc[indices[0][1:]]

[[2.10734243e-08 1.17709224e-01 6.03835284e-01 7.28952554e-01
  9.42354599e-01 9.43965112e-01]] [[  2471 184659  14559  46390  68417  68416]]


,title
198823,Queen of the Damned (Vampire Chronicles (Paper...
15190,"Queen of the Damned (Vampire Chronicles, Book ..."
49078,Queen of the Damned
72842,Interview With the Vampire (Vampire Chronicles...
72841,The Vampire Lestat (Vampire Chronicles (Paperb...


In [66]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  recommended_books = [book, []]

  query_row = df_joined[df_joined['title'] == 'The Queen of the Damned (Vampire Chronicles (Paperback))'].iloc[0]
  X_query_title = vectorizer_title.transform([query_row['title']])
  X_query_author = vectorizer_author.transform([query_row['author']])
  X_query = hstack([X_query_title, X_query_author])
  distances, indices = skl_nn.kneighbors(X_query, return_distance=True)
  titles = df_joined['title'].iloc[indices[0][1:]]

  for t, d in zip(titles, distances[0]):
    recommended_books[1].append([t, d])

  return recommended_books

In [67]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['Queen of the Damned (Vampire Chronicles (Paperback))', np.float64(2.1073424255447017e-08)], ['Queen of the Damned (Vampire Chronicles, Book III)', np.float64(0.1177092236706443)], ['Queen of the Damned', np.float64(0.6038352843530093)], ['Interview With the Vampire (Vampire Chronicles (Paperback))', np.float64(0.728952553647767)], ['The Vampire Lestat (Vampire Chronicles (Paperback))', np.float64(0.9423545989554771)]]]
You haven't passed yet. Keep trying!
